In [17]:
import time
from klampt import *
from IPython.display import clear_output
import ipywidgets as widgets
import ipyklampt

kvis = ipyklampt.KlamptWidget()
playback_widget = ipyklampt.Playback(kvis)

#If you'd like to show the print output, comment out this line
#clear_output()
display(kvis)
display(playback_widget)

drive_button = None
layout = widgets.Layout(width='30px', height='30px')
left = widgets.Button(description='',icon='toggle-left', layout=layout)
right = widgets.Button(description='',icon='toggle-right', layout=layout)
up = widgets.Button(description='',icon='toggle-up', layout=layout)
down = widgets.Button(description='',icon='toggle-down', layout=layout)
stop = widgets.Button(description='',icon='hand-stop-o', layout=layout)
def driveset(b):
    global drive_button
    if b is left: drive_button = 'left'
    elif b is right: drive_button = 'right'
    elif b is up: drive_button = 'up'
    elif b is down: drive_button = 'down'
    elif b is stop: drive_button = None
    else: print "Invalid drive button",b
left.on_click(driveset)
right.on_click(driveset)
up.on_click(driveset)
down.on_click(driveset)
stop.on_click(driveset)
display(widgets.HBox([left,right,down,up,stop]))

#Controls:
#left mouse click to rotate the view
#right click or ctrl+click to pan the view
#mouse wheel or shift+click to zoom the view

KlamptWidget()

UGxheWJhY2soY2hpbGRyZW49KEhCb3goY2hpbGRyZW49KEJ1dHRvbihkZXNjcmlwdGlvbj11J1BsYXknLCBpY29uPXUncGxheScsIHN0eWxlPUJ1dHRvblN0eWxlKCksIHRvb2x0aXA9dSdTdGHigKY=


SEJveChjaGlsZHJlbj0oQnV0dG9uKGljb249dSd0b2dnbGUtbGVmdCcsIGxheW91dD1MYXlvdXQoaGVpZ2h0PXUnMzBweCcsIHdpZHRoPXUnMzBweCcpLCBzdHlsZT1CdXR0b25TdHlsZSgpKSzigKY=


In [18]:
########################## Your code goes in this cell ################################
from klampt.math import vectorops,so2,so3,se3
import math
import random

def get_rotation_matrix(xform):
    """TODO: return the 2x2 rotation matrix corresponding to the given xform.
    In:
    - xform: a tuple (tx,ty,angle) indicating the translation amount (tx,ty)
      and the rotation amount angle (in degrees)
    Out:
    - R: a list of lists, indicating the rotation matrix corresponding to the
      given angle. The entry R[i][j] is the i'th row and j'th column
      (zero based). 
    """
    #TODO: fill in your entries here
    return so2.matrix(math.radians(xform[2]))
    a,b,c,d=1,0,0,1
    return [[a,b],
            [c,d]]

def get_translation_vector(xform):
    """Return the 2d translation vector corresponding to the given xform.
    In:
    - xform: a tuple (tx,ty,angle) indicating the translation amount (tx,ty)
      and the rotation amount angle (in degrees)
    Out:
    - the translation vector (tx,ty)
    """
    return [xform[0],xform[1]]

def lab1c_apply_xform(xform,point):
    """Apply a 2D rigid transform to a 2D point P.
    In:
    - xform: a tuple (tx,ty,angle) indicating the translation amount (tx,ty)
      and the rotation amount angle (in degrees)
    - point: a pair (px,py) indicating the coordinates of P in the original
      frame.
    Out:
    - a pair (qx,qy) indicating the coordinates of P after rotating by
      angle and then translating by (tx,ty). 
    """
    (tx,ty,angle) = xform
    t = get_translation_vector(xform)
    R = get_rotation_matrix(xform)
    Rp = [vectorops.dot(R[0],point),vectorops.dot(R[1],point)]
    return vectorops.add(Rp,t)

def lab1c(xform1,xform2):
    """TODO: compose two 2D transforms.  In other words, performs the operation
    xform1*xform2, where xform = lab1c(xform1,xform2) = xform1*xform2 ,
    is the transform that satisfies
      lab1c_apply_xform(xform,p) = lab1c_apply_xform(xform1,apply_xform(xform2,p))
    for all points p
    
    In:
     - xform1: a tuple (tx1,ty1,angle1) (angles in degrees)
     - xform2: a tuple (tx2,ty2,angle2) (angles in degrees)
    Out:
     - xform: a tuple (tx,ty,angle) representing the output transform
     """
    return vectorops.add(xform1,xform2)
    return xform2

def fuzzy_eq(a,b,eps=1e-8):
    """Returns true if a is within +/- eps of b."""
    return abs(a-b)<=eps

def fuzzy_veq(a,b,eps=1e-8):
    return all(fuzzy_eq(ai,bi,eps) for ai,bi in zip(a,b)) 

def selfTest():
    """You may use this function to make sure your values are correct"""
    R=get_rotation_matrix((0,0,0))
    assert fuzzy_veq(R[0],[1,0])
    assert fuzzy_veq(R[1],[0,1])
    R=get_rotation_matrix((0,0,90))
    assert fuzzy_veq(R[0],[0,-1])
    assert fuzzy_veq(R[1],[1,0])
    R=get_rotation_matrix((0,0,180))
    assert fuzzy_veq(R[0],[-1,0])
    assert fuzzy_veq(R[1],[0,-1])
    assert fuzzy_veq(lab1c((0,0,0),(20,30,70)),(20,30,70))
    assert fuzzy_veq(lab1c((-40,-5,0),(20,30,70)),(-20,25,70))
    assert fuzzy_veq(lab1c((0,0,180),(20,30,70)),(-20,-30,250))
    return

def lab1c_control():
    return random.choice(['left','right','up','down'])


In [19]:
#run this cell to bind your code to the visualization playback window

world = WorldModel()
car_body = None
#fl, fr, bl, br
wheels = {'fl':None,'fr':None,'bl':None,'br':None}
velocity = 0
steer = 0
vinc = 0.04
vmin = -1
vmax = 1
steermax = 45
steerinc = 2
xform = [0,0,0]
random_control = True

def boilerplate_start():
    global world,car_body,wheels,velocity,xform,random_control
    world = WorldModel()
    car_body = world.loadElement("data/body.obj")
    wheels['fl'] = world.loadElement("data/tire_fl.obj")
    wheels['fr'] = world.loadElement("data/tire_fr.obj")
    wheels['bl'] = world.loadElement("data/tire_bl.obj")
    wheels['br'] = world.loadElement("data/tire_br.obj")
    car_body = world.rigidObject(car_body)
    car_body.appearance().setColor(1,0,0,1)
    for (k,v) in wheels.iteritems():
        wheels[k] = world.rigidObject(v)
        flipzy = so3.rotation([1,0,0],math.pi/2)
        wheels[k].geometry().transform(flipzy,[0,0,0])
        wheels[k].appearance().setColor(0.3,0.3,0.3,1)
    velocity = 0
    xform = [0,0,0]
    update_car()
    kvis.set_world(world)

def update_2d_xform(rigidObject,xform2d):
    """Given an RigidObjectModel and a 2D transform (x,y,theta), sets the object's 3D transform
    while keeping height constant"""
    xform3d = rigidObject.getTransform()
    R,t = xform3d
    x,y,theta = xform2d
    theta = math.radians(theta)
    t = [x,y,t[2]]
    R = so3.rotation([0,0,1],theta)
    rigidObject.setTransform(R,t)

def update_car():
    global xform,steer
    steeringAngle = math.radians(steer)
    frontTirePos1 = (0.50,0.16)
    frontTirePos2 = (0.50,-0.16)
    rearTirePos1 = (0,0.16)
    rearTirePos2 = (0,-0.16)
    rearPoint1 = lab1c_apply_xform(xform,rearTirePos1)
    rearPoint2 = lab1c_apply_xform(xform,rearTirePos2)
    update_2d_xform(wheels['bl'],(rearPoint1[0],rearPoint1[1],xform[2]))
    update_2d_xform(wheels['br'],(rearPoint2[0],rearPoint2[1],xform[2]))
    tire1xform = lab1c(xform,(frontTirePos1[0],frontTirePos1[1],steer))
    tire2xform = lab1c(xform,(frontTirePos2[0],frontTirePos2[1],steer))
    update_2d_xform(wheels['fl'],tire1xform)
    update_2d_xform(wheels['fr'],tire2xform)
    update_2d_xform(car_body,xform)
    return

def update_xform(dt=0.02):
    global xform,velocity,steer
    rangle = math.radians(xform[2])
    v = [velocity*math.cos(rangle),velocity*math.sin(rangle)]
    dtheta = velocity*steer*5
    xform = (xform[0]+v[0]*dt,xform[1]+v[1]*dt,xform[2]+dtheta*dt)

def boilerplate_advance():
    global random_control
    update_xform()
    update_car()
    if random_control:
        if drive_button is None:
            u = lab1c_control()
            #print xform,u
            boilerplate_keypress(u)
        else:
            boilerplate_keypress(drive_button)
    #boilerplate_keypress(random.choice(['up','down','left','right']))

def boilerplate_keypress(c):
    global velocity,steer,xform,random_control
    if c.startswith('Arrow'):
        random_control = False
        boilerplate_keypress(c[5:].lower())
    elif c=='w':
        random_control = False
        boilerplate_keypress('up')
    elif c=='a':
        random_control = False
        boilerplate_keypress('left')
    elif c=='s':
        random_control = False
        boilerplate_keypress('down')
    elif c=='d':
        random_control = False
        boilerplate_keypress('right')
    elif c=='up':
        velocity += vinc
        if velocity > vmax:
            velocity = vmax
    elif c=='down':
        velocity -= vinc
        if velocity < vmin:
            velocity = vmin
    elif c=='right':
        steer -= steerinc
        if steer < -steermax:
            steer = -steermax
    elif c=='left':
        steer += steerinc
        if steer > steermax:
            steer = steermax
    elif c=='r':
        random_control = True

boilerplate_start()
playback_widget.quiet = False
playback_widget.reset = boilerplate_start
playback_widget.advance = boilerplate_advance

In [ ]:
#testing... run this cell to verify your code
selfTest()